In [1]:
# import sys
# sys.path.append("..")
# from common_utils import DATA_HOME
import pandas as pd
import sklearn
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer, classification_report, confusion_matrix
from collections import Counter
import xgboost as xgb
# dataset = "playground-series-s4e2"


In [2]:
# DATA_HOME
train_data = pd.read_csv("/Users/anyxling/datasets/playground-series-s4e2/train.csv", index_col=0)
# print(train_data.describe())
train_data.shape
# set(train_data["CALC"])

(20758, 17)

In [17]:
train_data.head()

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
id,,,,,,,,,,,,,,,,,
0,Male,24.443011,1.699998,81.669950,yes,yes,2.000000,2.983297,Sometimes,no,2.763573,no,0.000000,0.976473,Sometimes,Public_Transportation,Overweight_Level_II
1,Female,18.000000,1.560000,57.000000,yes,yes,2.000000,3.000000,Frequently,no,2.000000,no,1.000000,1.000000,no,Automobile,Normal_Weight
2,Female,18.000000,1.711460,50.165754,yes,yes,1.880534,1.411685,Sometimes,no,1.910378,no,0.866045,1.673584,no,Public_Transportation,Insufficient_Weight
3,Female,20.952737,1.710730,131.274851,yes,yes,3.000000,3.000000,Sometimes,no,1.674061,no,1.467863,0.780199,Sometimes,Public_Transportation,Obesity_Type_III
4,Male,31.641081,1.914186,93.798055,yes,yes,2.679664,1.971472,Sometimes,no,1.979848,no,1.967973,0.931721,Sometimes,Public_Transportation,Overweight_Level_II


In [16]:
# get the number of classes
train_data.iloc[:, -1].nunique()

7

In [3]:
test_data = pd.read_csv('/Users/anyxling/datasets/playground-series-s4e2/test.csv', index_col=0)
# print(test_data.describe())
test_data.shape
# test_data[test_data["CALC"]=="Always"]

(13840, 16)

In [4]:
train_test = pd.concat([train_data, test_data])
train_test.dtypes

Gender                             object
Age                               float64
Height                            float64
Weight                            float64
family_history_with_overweight     object
FAVC                               object
FCVC                              float64
NCP                               float64
CAEC                               object
SMOKE                              object
CH2O                              float64
SCC                                object
FAF                               float64
TUE                               float64
CALC                               object
MTRANS                             object
NObeyesdad                         object
dtype: object

In [5]:
train_test.describe()

,Age,Height,Weight,FCVC,NCP,CH2O,FAF,TUE
count,34598.000000,34598.000000,34598.000000,34598.000000,34598.000000,34598.000000,34598.000000,34598.000000
mean,23.886181,1.699721,87.686451,2.444704,2.757043,2.030469,0.978861,0.614467
std,5.733207,0.087895,26.273493,0.532568,0.707610,0.609566,0.839122,0.604475
min,14.000000,1.450000,39.000000,1.000000,1.000000,1.000000,0.000000,0.000000
25%,20.000000,1.631856,66.000000,2.000000,3.000000,1.784710,0.006892,0.000000
50%,22.851747,1.700000,84.000000,2.392179,3.000000,2.000000,1.000000,0.555591
75%,26.000000,1.761773,111.539494,3.000000,3.000000,2.550570,1.583832,1.000000
max,61.000000,1.980000,165.057269,3.000000,4.000000,3.000000,3.000000,2.000000


In [6]:
# check outliers
outliers = []
for col in train_test.select_dtypes(include='float').columns:
    col_mean = train_test[col].mean()
    col_std = train_test[col].std()
    col_outliers = train_test[(train_test[col] > col_mean + 3 * col_std) | (train_test[col] < col_mean - 3 * col_std)]
    for idx in col_outliers.index:
        outliers.append((col, idx))

print(outliers)

[('Age', 25), ('Age', 56), ('Age', 90), ('Age', 190), ('Age', 310), ('Age', 472), ('Age', 510), ('Age', 526), ('Age', 582), ('Age', 584), ('Age', 667), ('Age', 862), ('Age', 930), ('Age', 948), ('Age', 1222), ('Age', 1401), ('Age', 1524), ('Age', 1557), ('Age', 1692), ('Age', 1787), ('Age', 1806), ('Age', 2006), ('Age', 2195), ('Age', 2216), ('Age', 2355), ('Age', 2538), ('Age', 2553), ('Age', 2571), ('Age', 2590), ('Age', 2642), ('Age', 2664), ('Age', 2797), ('Age', 2834), ('Age', 3174), ('Age', 3183), ('Age', 3205), ('Age', 3284), ('Age', 3378), ('Age', 3471), ('Age', 3495), ('Age', 3747), ('Age', 3893), ('Age', 3929), ('Age', 3995), ('Age', 4351), ('Age', 4395), ('Age', 4698), ('Age', 4805), ('Age', 5261), ('Age', 5301), ('Age', 5303), ('Age', 5389), ('Age', 5423), ('Age', 5476), ('Age', 5646), ('Age', 5711), ('Age', 5731), ('Age', 5769), ('Age', 5822), ('Age', 5935), ('Age', 5965), ('Age', 6060), ('Age', 6311), ('Age', 6388), ('Age', 6401), ('Age', 6415), ('Age', 7137), ('Age', 722

In [7]:
# handle outliers 
for outlier in outliers:
    col, idx = outlier
    val = train_test[col][idx]
    col_mean = train_test[col].mean()
    col_std = train_test[col].std()
    if val > col_mean + 3 * col_std:
        train_test[col][idx] = col_mean + 3 * col_std
    if val < col_mean - 3 * col_std:
        train_test[col][idx] = col_mean - 3 * col_std

/var/folders/h0/hz2hnl6s6z710v5lcgqmt9xm0000gn/T/ipykernel_19471/3382365019.py:8: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  train_test[col][idx] = col_mean + 3 * col_std
/var/folders/h0/hz2hnl6s6z710v5lcgqmt9xm0000gn/T/ipykernel_19471/33

In [8]:
train_test.describe()

,Age,Height,Weight,FCVC,NCP,CH2O,FAF,TUE
count,34598.000000,34598.000000,34598.000000,34598.000000,34598.000000,34598.000000,34598.000000,34598.000000
mean,23.817278,1.699718,87.686451,2.444704,2.757043,2.030469,0.978861,0.614467
std,5.456280,0.087887,26.273493,0.532568,0.707610,0.609566,0.839122,0.604475
min,14.000000,1.450000,39.000000,1.000000,1.000000,1.000000,0.000000,0.000000
25%,20.000000,1.631856,66.000000,2.000000,3.000000,1.784710,0.006892,0.000000
50%,22.851747,1.700000,84.000000,2.392179,3.000000,2.000000,1.000000,0.555591
75%,26.000000,1.761773,111.539494,3.000000,3.000000,2.550570,1.583832,1.000000
max,41.085801,1.963407,165.057269,3.000000,4.000000,3.000000,3.000000,2.000000


In [9]:
# scale the data
sc = StandardScaler()
num_cols = train_test.select_dtypes(include='float').columns
train_test[num_cols] = sc.fit_transform(train_test[num_cols])
train_test.describe()


,Age,Height,Weight,FCVC,NCP,CH2O,FAF,TUE
count,3.459800e+04,3.459800e+04,3.459800e+04,3.459800e+04,3.459800e+04,3.459800e+04,3.459800e+04,3.459800e+04
mean,-4.191623e-16,2.172004e-15,-2.330961e-17,-2.873141e-16,6.074875e-16,-7.136644e-16,-1.887360e-16,-1.015560e-16
std,1.000014e+00,1.000014e+00,1.000014e+00,1.000014e+00,1.000014e+00,1.000014e+00,1.000014e+00,1.000014e+00
min,-1.799288e+00,-2.841395e+00,-1.853090e+00,-2.712750e+00,-2.483103e+00,-1.690520e+00,-1.166547e+00,-1.016544e+00
25%,-6.996219e-01,-7.721605e-01,-8.254237e-01,-8.350300e-01,3.433533e-01,-4.031756e-01,-1.158333e+00,-1.016544e+00
50%,-1.769603e-01,3.210610e-03,-1.403127e-01,-9.862752e-02,3.433533e-01,-4.998488e-02,2.519247e-02,-9.740132e-02
75%,4.000442e-01,7.060898e-01,9.078879e-01,1.042690e+00,3.433533e-01,8.532444e-01,7.209682e-01,6.378077e-01
max,3.164935e+00,3.000362e+00,2.944867e+00,1.042690e+00,1.756581e+00,1.590550e+00,2.408672e+00,2.292160e+00


In [10]:
# encode categorical features
str_cols = train_test.iloc[:, :-1].select_dtypes(include=['object']).columns
le_features = LabelEncoder()
for col in str_cols:
    train_test[col] = le_features.fit_transform(train_test[col])
    print(col, dict(zip(le_features.classes_, range(len(le_features.classes_)))))

Gender {'Female': 0, 'Male': 1}
family_history_with_overweight {'no': 0, 'yes': 1}
FAVC {'no': 0, 'yes': 1}
CAEC {'Always': 0, 'Frequently': 1, 'Sometimes': 2, 'no': 3}
SMOKE {'no': 0, 'yes': 1}
SCC {'no': 0, 'yes': 1}
CALC {'Always': 0, 'Frequently': 1, 'Sometimes': 2, 'no': 3}
MTRANS {'Automobile': 0, 'Bike': 1, 'Motorbike': 2, 'Public_Transportation': 3, 'Walking': 4}


In [11]:
# split train and test data
test_encoded = train_test[pd.isna(train_test['NObeyesdad'])]
train_encoded = train_test[pd.notna(train_test['NObeyesdad'])]

# encode labels in train data
le_labels = LabelEncoder()
y_train_encoded = le_labels.fit_transform(train_encoded.iloc[:, -1])
print(len(y_train_encoded))
# train_encoded.iloc[:, -1] = le_labels.fit_transform(train_encoded.iloc[:, -1])
# train_encoded.iloc[:, -1] = train_encoded.iloc[:, -1].astype(int)
# print(train_encoded.iloc[:, -1].dtype)

20758


In [12]:
# check if train data is imbalanced or not
label_dist = Counter(y_train_encoded)
print(label_dist)

Counter({4: 4046, 3: 3248, 1: 3082, 2: 2910, 0: 2523, 6: 2522, 5: 2427})


In [13]:
X_train, X_val_test, y_train, y_val_test = train_test_split(train_encoded.iloc[:, :-1], y_train_encoded, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_val_test, y_val_test, test_size=0.5, random_state=42)
X_train.shape, X_val.shape, X_test.shape

((16606, 16), (2076, 16), (2076, 16))

In [20]:
clf = xgb.XGBClassifier(objective='multi:softmax', num_class=7, eval_metric='mlogloss')
clf.fit(X_train, y_train, verbose=True, eval_set=[(X_val, y_val)], early_stopping_rounds=10)

[0]	validation_0-mlogloss:1.24327
[1]	validation_0-mlogloss:0.96112
[2]	validation_0-mlogloss:0.78347
[3]	validation_0-mlogloss:0.66159
[4]	validation_0-mlogloss:0.57274
[5]	validation_0-mlogloss:0.50835


/Users/anyxling/python-general/lib/python3.12/site-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[6]	validation_0-mlogloss:0.45868
[7]	validation_0-mlogloss:0.42078
[8]	validation_0-mlogloss:0.39231
[9]	validation_0-mlogloss:0.37078
[10]	validation_0-mlogloss:0.35235
[11]	validation_0-mlogloss:0.33737
[12]	validation_0-mlogloss:0.32575
[13]	validation_0-mlogloss:0.31636
[14]	validation_0-mlogloss:0.30873
[15]	validation_0-mlogloss:0.30339
[16]	validation_0-mlogloss:0.29821
[17]	validation_0-mlogloss:0.29413
[18]	validation_0-mlogloss:0.29115
[19]	validation_0-mlogloss:0.28855
[20]	validation_0-mlogloss:0.28517
[21]	validation_0-mlogloss:0.28327
[22]	validation_0-mlogloss:0.28085
[23]	validation_0-mlogloss:0.28001
[24]	validation_0-mlogloss:0.27904
[25]	validation_0-mlogloss:0.27777
[26]	validation_0-mlogloss:0.27623
[27]	validation_0-mlogloss:0.27473
[28]	validation_0-mlogloss:0.27420
[29]	validation_0-mlogloss:0.27419
[30]	validation_0-mlogloss:0.27356
[31]	validation_0-mlogloss:0.27243
[32]	validation_0-mlogloss:0.27214
[33]	validation_0-mlogloss:0.27189
[34]	validation_0-mloglo

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_class=7, num_parallel_tree=None, ...)

In [22]:
pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, pred)
print("accuracy score for test data:", accuracy)

accuracy score for test data: 0.9022157996146436


In [23]:
# compare the validation accuracy with the baseline model
dummy_clf = DummyClassifier(strategy='most_frequent') # naive classifier, majority vote
dummy_clf.fit(X_train, y_train)
y_pred_dummy = dummy_clf.predict(X_test)
accuracy_dummy = accuracy_score(y_test, y_pred_dummy)
print("accuracy score for dummy classifier:", accuracy_dummy)

accuracy score for dummy classifier: 0.1960500963391137


In [24]:
y_pred = clf.predict(test_encoded.iloc[:, :-1])

In [25]:
decoded_pred = le_labels.inverse_transform(y_pred)
decoded_pred.shape

(13840,)

In [26]:
test_data_idx = pd.read_csv('/Users/anyxling/datasets/playground-series-s4e2/test.csv')
sub = pd.DataFrame({
    "id": test_data_idx['id'],
    "NObeyesdad": decoded_pred
})
sub

,id,NObeyesdad
0,20758,Obesity_Type_II
1,20759,Overweight_Level_I
2,20760,Obesity_Type_III
3,20761,Obesity_Type_I
4,20762,Obesity_Type_III
...,...,...
13835,34593,Overweight_Level_II
13836,34594,Overweight_Level_I
13837,34595,Insufficient_Weight
13838,34596,Normal_Weight


In [27]:
sub.to_csv('submission.csv', index=False)